In [5]:
import pandas as pd
import math
import numpy as np
import random
import sqlite3
import time
from os.path import expanduser

In [6]:
CONVERTED_DATABASE_NAME = '/Users/darylwilding-mcbride/Downloads/experiments/P3856/converted-databases/exp-P3856-run-P3856_YHE211_1_Slot1-1_1_5104-converted.sqlite'

In [9]:
start_run = time.time()

db_conn = sqlite3.connect(CONVERTED_DATABASE_NAME)
raw_df = pd.read_sql_query("SELECT frame_id,mz,retention_time_secs,scan,intensity FROM frames where retention_time_secs >= 1650 and retention_time_secs <= 1750", db_conn)
db_conn.close()

stop_run = time.time()
print("total running time: {} seconds".format(round(stop_run-start_run,1)))

total running time: 324.8 seconds


In [7]:
HDF_DATABASE_NAME = '{}/db.hdf'.format(expanduser('~'))
HDF_DATABASE_NAME

'/Users/darylwilding-mcbride/db.hdf'

In [12]:
raw_df.to_hdf(HDF_DATABASE_NAME, key='frames', format='table', data_columns=True)

In [13]:
start_run = time.time()
raw_df = pd.read_hdf(HDF_DATABASE_NAME, key='frames', columns=['frame_id','mz','retention_time_secs','scan','intensity'], where=['retention_time_secs >= {} and retention_time_secs <= {}'.format(1650, 1750)])
stop_run = time.time()
print("total running time: {} seconds".format(round(stop_run-start_run,1)))

total running time: 24.7 seconds


In [13]:
raw_df.sample(n=3)

,frame_id,mz,retention_time_secs,scan,intensity
35576838,15327,679.305186,1726.129869,732,73
32291317,15262,1020.428067,1719.246419,335,20
8613329,14782,716.334464,1668.437367,160,22


In [8]:
start_run = time.time()
for i in range(200):
    subset_df = raw_df[(raw_df.mz >= 700) & (raw_df.mz <= 720) & (raw_df.scan >= 500) & (raw_df.scan <= 550) & (raw_df.retention_time_secs >= 1700) & (raw_df.retention_time_secs <= 1760)]
stop_run = time.time()
print("total running time: {} seconds".format(round(stop_run-start_run,1)))

total running time: 78.8 seconds


In [13]:
start_run = time.time()
for i in range(10000):
    rt_df = subset_df.groupby(['frame_id'], as_index=False).intensity.sum()
stop_run = time.time()
print("total running time: {} seconds".format(round(stop_run-start_run,1)))

total running time: 16.0 seconds
